In [3]:
import os
import boto3
from botocore.exceptions import ClientError
from dotenv import load_dotenv
import pandas as pd
import psycopg2 as psycopg
import mlflow
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import (
    OneHotEncoder, 
    SplineTransformer, 
    QuantileTransformer, 
    RobustScaler,
    PolynomialFeatures,
    KBinsDiscretizer,
)

In [2]:
# Загрузка переменных из .env файла
load_dotenv()

# Функция для получения переменных окружения
def get_env_variable(var_name):
    value = os.getenv(var_name)
    if not value:
        raise ValueError(f"Переменная окружения {var_name} не установлена в файле .env")
    return value

# Получение переменных окружения
S3_ENDPOINT_URL = get_env_variable('S3_ENDPOINT_URL')
S3_BUCKET_NAME = get_env_variable('S3_BUCKET_NAME')
AWS_ACCESS_KEY_ID = get_env_variable('AWS_ACCESS_KEY_ID')
AWS_SECRET_ACCESS_KEY = get_env_variable('AWS_SECRET_ACCESS_KEY')

# Создание клиента S3
s3 = boto3.client(
    's3',
    endpoint_url=S3_ENDPOINT_URL,
    aws_access_key_id=AWS_ACCESS_KEY_ID,
    aws_secret_access_key=AWS_SECRET_ACCESS_KEY
)

def list_s3_objects():
    try:
        # Получение списка объектов в бакете
        response = s3.list_objects_v2(Bucket=S3_BUCKET_NAME)

        # Проверка, пуст ли бакет
        if 'Contents' not in response:
            print(f"Бакет '{S3_BUCKET_NAME}' пуст.")
            return

        # Вывод списка объектов
        print(f"Объекты в бакете '{S3_BUCKET_NAME}':")
        for obj in response['Contents']:
            print(f"- {obj['Key']} (Размер: {obj['Size']} байт, Последнее изменение: {obj['LastModified']})")

    except ClientError as e:
        print(f"Произошла ошибка: {e}")

# Вызов функции для вывода списка объектов
list_s3_objects()


Объекты в бакете 's3-student-mle-20241121-573777dca2':
- 0/287581d0c77a401b8a2bc3eeaad40cfe/artifacts/models/MLmodel (Размер: 484 байт, Последнее изменение: 2025-08-03 13:22:47.306000+00:00)
- 0/287581d0c77a401b8a2bc3eeaad40cfe/artifacts/models/conda.yaml (Размер: 207 байт, Последнее изменение: 2025-08-03 13:22:47.335000+00:00)
- 0/287581d0c77a401b8a2bc3eeaad40cfe/artifacts/models/model.cb (Размер: 63904 байт, Последнее изменение: 2025-08-03 13:22:47.237000+00:00)
- 0/287581d0c77a401b8a2bc3eeaad40cfe/artifacts/models/python_env.yaml (Размер: 115 байт, Последнее изменение: 2025-08-03 13:22:47.264000+00:00)
- 0/287581d0c77a401b8a2bc3eeaad40cfe/artifacts/models/requirements.txt (Размер: 89 байт, Последнее изменение: 2025-08-03 13:22:47.365000+00:00)
- 0/2b4d7f751a8a451d8bf27e6214ebc988/artifacts/comment.txt (Размер: 89 байт, Последнее изменение: 2025-08-02 12:14:54.784000+00:00)
- 0/4221086470a747c080d599575d568fd7/artifacts/data_description.txt (Размер: 35 байт, Последнее изменение: 2025

In [45]:
TRACKING_SERVER_HOST = "127.0.0.1"
TRACKING_SERVER_PORT = 5000

EXPERIMENT_NAME = "churn_ivan_panchenko" # название эксперимента
RUN_NAME = "model_0_registry" 
REGISTRY_MODEL_NAME = "churn_model_ivan_panchenko" # название зарегистрированной модели 


connection = {"sslmode": "require", "target_session_attrs": "read-write"}
postgres_credentials = {
    "host": os.getenv('DB_DESTINATION_HOST'),
    "port": os.getenv('DB_DESTINATION_PORT'), 
    "dbname": os.getenv('DB_DESTINATION_NAME'), 
    "user": os.getenv('DB_DESTINATION_USER'), 
    "password": os.getenv('DB_DESTINATION_PASSWORD')
}
assert all([var_value != "" for var_value in list(postgres_credentials.values())])

connection.update(postgres_credentials)

TABLE_NAME = "users_churn"


with psycopg.connect(**connection) as conn:

    with conn.cursor() as cur:
        cur.execute(f"SELECT * FROM {TABLE_NAME}")
				
        data = cur.fetchall()

        columns = [col[0] for col in cur.description]

df = pd.DataFrame(data, columns=columns)

print(f"Размер нашей таблицы: {df.shape[0]} строк; {df.shape[1]} столбцов")

Размер нашей таблицы: 7043 строк; 22 столбцов


In [46]:
# определение категориальных колонок, которые будут преобразованы
cat_columns = ["type", "payment_method", "internet_service", "gender"]

# создание объекта OneHotEncoder для преобразования категориальных переменных
# auto - автоматическое определение категорий
# ignore - игнорировать ошибки, если встречается неизвестная категория
# max_categories - максимальное количество уникальных категорий
# sparse_output - вывод в виде разреженной матрицы, если False, то в виде обычного массива
# drop="first" - удаляет первую категорию, чтобы избежать ловушки мультиколлинеарности
encoder_oh = OneHotEncoder(categories='auto', handle_unknown='ignore', max_categories=10, sparse_output=False, drop='first')

# применение OneHotEncoder к данным. Преобразование категориальных данных в массив

encoded_features = encoder_oh.fit_transform(df[cat_columns].to_numpy())

# преобразование полученных признаков в DataFrame и установка названий колонок
# get_feature_names_out() - получение имён признаков после преобразования
encoded_df = pd.DataFrame(encoded_features, columns=encoder_oh.get_feature_names_out(cat_columns))

# конкатенация исходного DataFrame с новым DataFrame, содержащим закодированные категориальные признаки
# axis=1 означает конкатенацию по колонкам
obj_df = pd.concat([df, encoded_df], axis=1)

obj_df.head(2)

,id,customer_id,begin_date,end_date,type,paperless_billing,payment_method,monthly_charges,total_charges,internet_service,...,multiple_lines,target,type_One year,type_Two year,payment_method_Credit card (automatic),payment_method_Electronic check,payment_method_Mailed check,internet_service_Fiber optic,internet_service_None,gender_Male
0,17,8191-XWSZG,2015-10-01,NaT,One year,No,Mailed check,20.65,1022.95,None,...,No,0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
1,59,3957-SQXML,2017-04-01,NaT,Two year,No,Credit card (automatic),24.95,894.30,None,...,Yes,0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0


In [47]:
from sklearn.impute import SimpleImputer

num_columns = ["monthly_charges", "total_charges"]

n_knots = 3
degree_spline = 4
n_quantiles = 100
degree = 3
n_bins = 5
encode = 'ordinal'
strategy = 'uniform'
subsample = None

# Предобработка данных для устранения NaN
imputer = SimpleImputer(strategy='mean')
df[num_columns] = imputer.fit_transform(df[num_columns])

# SplineTransformer
encoder_spl = SplineTransformer(n_knots=n_knots, degree=degree_spline)
encoded_features = encoder_spl.fit_transform(df[num_columns].to_numpy())
encoded_df = pd.DataFrame(
    encoded_features,
    columns=encoder_spl.get_feature_names_out(num_columns)
)
num_df = pd.concat([num_df, encoded_df], axis=1)

# QuantileTransformer
encoder_q = QuantileTransformer(n_quantiles=n_quantiles)
encoded_features = encoder_q.fit_transform(df[num_columns].to_numpy())
encoded_df = pd.DataFrame(encoded_features, columns=encoder_q.get_feature_names_out(num_columns))
encoded_df.columns = [col + f"_q_{n_quantiles}" for col in num_columns]
num_df = pd.concat([num_df, encoded_df], axis=1)

# RobustScaler
encoder_rb = RobustScaler()
encoded_features = encoder_rb.fit_transform(df[num_columns].to_numpy())
encoded_df = pd.DataFrame(encoded_features, columns=encoder_rb.get_feature_names_out(num_columns))
encoded_df.columns = [col + f'_robust' for col in num_columns]
num_df = pd.concat([num_df, encoded_df], axis=1)

# PolynomialFeatures
encoder_pol = PolynomialFeatures(degree=degree)
encoded_features = encoder_pol.fit_transform(df[num_columns].to_numpy())
encoded_df = pd.DataFrame(encoded_features, columns=encoder_pol.get_feature_names_out(num_columns))
# get all columns after the intercept and original features
encoded_df = encoded_df.iloc[:, 1 + len(num_columns):]
encoded_df.columns = [col + "_poly" for col in encoded_df.columns]
num_df = pd.concat([num_df, encoded_df], axis=1)

# KBinsDiscretizer
encoder_kbd = KBinsDiscretizer(n_bins=n_bins, encode=encode, strategy=strategy, subsample=subsample)
encoded_features = encoder_kbd.fit_transform(df[num_columns].to_numpy())
encoded_df = pd.DataFrame(encoded_features, columns=encoder_kbd.get_feature_names_out(num_columns))
encoded_df.columns = [col + f'_bin' for col in num_columns]
num_df = pd.concat([num_df, encoded_df], axis=1)

num_df.head(2)

,monthly_charges,total_charges,monthly_charges_sp_0,monthly_charges_sp_1,monthly_charges_sp_2,monthly_charges_sp_3,monthly_charges_sp_4,monthly_charges_sp_5,total_charges_sp_0,total_charges_sp_1,...,total_charges_robust,monthly_charges^2_poly,monthly_charges total_charges_poly,total_charges^2_poly,monthly_charges^3_poly,monthly_charges^2 total_charges_poly,monthly_charges total_charges^2_poly,total_charges^3_poly,monthly_charges_bin,total_charges_bin
0,20.65,1022.95,0.034259,0.433936,0.481590,0.050214,2.168151e-07,0.0,0.014515,0.334777,...,-0.111572,426.4225,21123.9175,1.046427e+06,8805.624625,436208.896375,2.160871e+07,1.070442e+09,0.0,0.0
1,24.95,894.30,0.023507,0.388355,0.519449,0.068676,1.316872e-05,0.0,0.016892,0.350947,...,-0.149584,622.5025,22312.7850,7.997725e+05,15531.437375,556703.985750,1.995432e+07,7.152365e+08,0.0,0.0


In [40]:
numeric_transformer = ColumnTransformer(
    transformers=[
        ('spl', encoder_spl, num_columns), 
        ('q', encoder_q, num_columns), 
        ('rb', encoder_rb, num_columns), 
        ('pol', encoder_pol, num_columns), 
        ('kbd', encoder_kbd, num_columns)
    ]
)

categorical_transformer = Pipeline(steps=[('encoder', encoder_oh)])

preprocessor = ColumnTransformer(
    transformers=[('num', numeric_transformer, num_columns), 
                  ('cat', categorical_transformer, cat_columns)], 
    n_jobs=-1)

encoded_features = preprocessor.fit_transform(df)

transformed_df = pd.DataFrame(
    encoded_features, 
    columns=preprocessor.get_feature_names_out()
)

df = pd.concat([df, transformed_df], axis=1)

df.head(2)

,id,customer_id,begin_date,end_date,type,paperless_billing,payment_method,monthly_charges,total_charges,internet_service,...,cat__streaming_tv_None,cat__streaming_movies_Yes,cat__streaming_movies_None,cat__gender_Male,cat__senior_citizen_1,cat__partner_Yes,cat__dependents_Yes,cat__multiple_lines_Yes,cat__multiple_lines_None,cat__target_1
0,17,8191-XWSZG,2015-10-01,NaT,One year,No,Mailed check,20.65,1022.95,None,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,59,3957-SQXML,2017-04-01,NaT,Two year,No,Credit card (automatic),24.95,894.30,None,...,1.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0


In [48]:
os.environ["MLFLOW_S3_ENDPOINT_URL"] = get_env_variable("S3_ENDPOINT_URL")
os.environ["AWS_ACCESS_KEY_ID"] = get_env_variable("AWS_ACCESS_KEY_ID")
os.environ["AWS_SECRET_ACCESS_KEY"] = get_env_variable("AWS_SECRET_ACCESS_KEY")


mlflow.set_tracking_uri(f"http://{TRACKING_SERVER_HOST}:{TRACKING_SERVER_PORT}")
mlflow.set_registry_uri(f"http://{TRACKING_SERVER_HOST}:{TRACKING_SERVER_PORT}")

experiment_id = mlflow.get_experiment_by_name(EXPERIMENT_NAME).experiment_id

with mlflow.start_run(run_name=RUN_NAME, experiment_id=experiment_id) as run:
    run_id = run.info.run_id

    mlflow.sklearn.log_model(preprocessor, "column_transformer") 

2025/08/08 12:07:07 WARNING mlflow.sklearn: Model was missing function: predict. Not logging python_function flavor!


In [49]:
preprocessor

ColumnTransformer(n_jobs=-1,
                  transformers=[('num',
                                 ColumnTransformer(transformers=[('spl',
                                                                  SplineTransformer(degree=4,
                                                                                    n_knots=3),
                                                                  ['monthly_charges',
                                                                   'total_charges']),
                                                                 ('q',
                                                                  QuantileTransformer(n_quantiles=100),
                                                                  ['monthly_charges',
                                                                   'total_charges']),
                                                                 ('rb',
                                                                  RobustScaler(),
                                                                  ['monthly_charges',
                                                                   'total_charges']),
                                                                 ('pol',
                                                                  PolynomialFeatures(degree=3),
                                                                  ['monthly_char...
                                                                handle_unknown='ignore',
                                                                max_categories=10,
                                                                sparse_output=False))]),
                                 ['id', 'customer_id', 'begin_date', 'end_date',
                                  'type', 'paperless_billing', 'payment_method',
                                  'internet_service', 'online_security',
                                  'online_backup', 'device_protection',
                                  'tech_support', 'streaming_tv',
                                  'streaming_movies', 'gender',
                                  'senior_citizen', 'partner', 'dependents',
                                  'multiple_lines', 'target'])])

In [50]:
run_id

'a93d67898d174c4f9a7b507324969e1b'

In [51]:
import os
import mlflow
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, roc_auc_score

In [62]:
model_registred_name=REGISTRY_MODEL_NAME

# Безопасное удаление колонок
columns_to_drop = ['id', 'customer_id', 'begin_date', 'end_date']
columns_to_drop = [col for col in columns_to_drop if col in df.columns]
df = df.drop(columns=columns_to_drop, errors='ignore')

# Определение категориальных и числовых колонок
cat_columns = df.select_dtypes(include=['object']).columns.tolist()
cat_columns = [col for col in cat_columns if col != 'target']
num_columns = df.select_dtypes(include=['float64', 'int64']).columns.tolist()

# One-hot кодирование категориальных признаков
df_encoded = pd.get_dummies(df, columns=cat_columns)

# Разделение данных
X_train, X_test, y_train, y_test = train_test_split(
    df_encoded.drop('target', axis=1), 
    df_encoded['target'], 
    test_size=0.2, 
    random_state=42
)

# Обучение модели
model = RandomForestClassifier(random_state=42)
model.fit(X_train, y_train)

# Оценка качества модели
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, model.predict_proba(X_test)[:, 1])

# Логирование метрик и модели в MLflow
with mlflow.start_run(run_name=RUN_NAME, experiment_id=experiment_id) as run:
    mlflow.log_metric("accuracy", accuracy)
    mlflow.log_metric("roc_auc", roc_auc)
    
    # Регистрация модели
    model_info = mlflow.sklearn.log_model(
        sk_model=model, 
        artifact_path="model", 
        registered_model_name=model_registred_name
    )
    
    # Получение версии модели
    client = mlflow.tracking.MlflowClient()
    model_version = client.get_latest_versions(model_registred_name)[0].version
    run_id = run.info.run_id
    model_version_id = model_version

print(f"Accuracy: {accuracy}")
print(f"ROC AUC: {roc_auc}")
print(f"Model Version ID: {model_version_id}")
print(f"Run ID: {run_id}")

Registered model 'churn_model_ivan_panchenko' already exists. Creating a new version of this model...
2025/08/08 12:24:00 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation. Model name: churn_model_ivan_panchenko, version 4


Accuracy: 0.7906316536550745
ROC AUC: 0.8232458793285953
Model Version ID: 4
Run ID: 31654e11cd674b778b4be251cbc80b80


Created version '4' of model 'churn_model_ivan_panchenko'.
